# US Airport Delays

This notebook will analyze the reasons for flight cancellations and delays in 2008 

[Airline Codes](http://stat-computing.org/dataexpo/2009/carriers.csv), [Airport Codes](http://stat-computing.org/dataexpo/2009/airports.csv), [Data Dictionary](http://stat-computing.org/dataexpo/2009/the-data.html)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
import sys
print sys.version
from matplotlib import style
style.use('fivethirtyeight')
%pylab inline

2.7.11 (default, Dec  5 2015, 14:44:47) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.1.76)]
Populating the interactive namespace from numpy and matplotlib


In [2]:
# read in data
flights = pd.read_csv('datasets/2008.csv.bz2')
#flights = pd.read_csv('http://stat-computing.org/dataexpo/2009/2008.csv.bz2', compression = 'bz2')
# 'bz2' compression is not supported in Python 2 :(

In [3]:
flights.columns

Index([u'Year', u'Month', u'DayofMonth', u'DayOfWeek', u'DepTime',
       u'CRSDepTime', u'ArrTime', u'CRSArrTime', u'UniqueCarrier',
       u'FlightNum', u'TailNum', u'ActualElapsedTime', u'CRSElapsedTime',
       u'AirTime', u'ArrDelay', u'DepDelay', u'Origin', u'Dest', u'Distance',
       u'TaxiIn', u'TaxiOut', u'Cancelled', u'CancellationCode', u'Diverted',
       u'CarrierDelay', u'WeatherDelay', u'NASDelay', u'SecurityDelay',
       u'LateAircraftDelay'],
      dtype='object')

In [4]:
# create lane pairs to determine highest traffic routes
flights['lanes'] = flights['Origin'] + ' - ' + flights['Dest']

# # create two empty lists to store origin airports and destination airports
# origin = []
# dest = []

# def split_col(series):
#     for i in series:
#         s = i.split('-') # split each pair on the dash
#         origin.append(s[0]) # append the first value to origin
#         dest.append(s[1]) # append the second value to destination
        
# # run split function on lane pair column in the merged dataframe
# split_col(flights['lanes'])

# # create new columns in merged df
# merged['Origin'] = np.asarray(origin)
# merged['Dest'] = np.asarray(dest)
# merged.head()

# next, let's read in some data on the airports and air carriers 
carriers = pd.read_csv('http://stat-computing.org/dataexpo/2009/carriers.csv')
airports = pd.read_csv('http://stat-computing.org/dataexpo/2009/airports.csv')

airports_orig = pd.DataFrame(airports[['iata','airport']])
airports_orig.rename(columns = {'iata':'Origin'}, inplace = True)

flights2 = pd.merge(flights,airports_orig, on ='Origin')
flights2.rename(columns = {'airport':'origin_airport'}, inplace = True)
flights2.columns

airports_dest = pd.DataFrame(airports[['iata','airport']])
airports_dest.rename(columns = {'iata':'Dest'}, inplace = True)

flights3 = pd.merge(flights2,airports_dest, on ='Dest')
flights3.rename(columns = {'airport':'dest_airport'}, inplace = True)

# then create origin-destination pairs
flights3['lane_pairs'] = flights3['origin_airport'] + '-' + flights3['dest_airport']

In [5]:
flights3.columns

Index([u'Year', u'Month', u'DayofMonth', u'DayOfWeek', u'DepTime',
       u'CRSDepTime', u'ArrTime', u'CRSArrTime', u'UniqueCarrier',
       u'FlightNum', u'TailNum', u'ActualElapsedTime', u'CRSElapsedTime',
       u'AirTime', u'ArrDelay', u'DepDelay', u'Origin', u'Dest', u'Distance',
       u'TaxiIn', u'TaxiOut', u'Cancelled', u'CancellationCode', u'Diverted',
       u'CarrierDelay', u'WeatherDelay', u'NASDelay', u'SecurityDelay',
       u'LateAircraftDelay', u'lanes', u'origin_airport', u'dest_airport',
       u'lane_pairs'],
      dtype='object')

In [6]:
# adding origin and destination state, airport codes
airports.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


In [7]:
# add origin state
orig_state = airports.rename(columns = {'iata':'Origin'})
orig_state2 = pd.merge(orig_state, flights3, on = 'Origin')
orig_state2.rename(columns = {'state':'orig_state'}, inplace = True)
#orig_state2.columns


# add destination state
dest_state = airports.rename(columns = {'iata':'Dest'})
dest_state2 = pd.merge(orig_state2,dest_state, on = 'Dest')
dest_state2.rename(columns = {'state':'dest_state'}, inplace = True)

dest_state2.columns

Index([u'Origin', u'airport_x', u'city_x', u'orig_state', u'country_x',
       u'lat_x', u'long_x', u'Year', u'Month', u'DayofMonth', u'DayOfWeek',
       u'DepTime', u'CRSDepTime', u'ArrTime', u'CRSArrTime', u'UniqueCarrier',
       u'FlightNum', u'TailNum', u'ActualElapsedTime', u'CRSElapsedTime',
       u'AirTime', u'ArrDelay', u'DepDelay', u'Dest', u'Distance', u'TaxiIn',
       u'TaxiOut', u'Cancelled', u'CancellationCode', u'Diverted',
       u'CarrierDelay', u'WeatherDelay', u'NASDelay', u'SecurityDelay',
       u'LateAircraftDelay', u'lanes', u'origin_airport', u'dest_airport',
       u'lane_pairs', u'airport_y', u'city_y', u'dest_state', u'country_y',
       u'lat_y', u'long_y'],
      dtype='object')

In [8]:
# rename fully merged dataframe for cleaner replication
flights5 = dest_state2
flights5.columns

Index([u'Origin', u'airport_x', u'city_x', u'orig_state', u'country_x',
       u'lat_x', u'long_x', u'Year', u'Month', u'DayofMonth', u'DayOfWeek',
       u'DepTime', u'CRSDepTime', u'ArrTime', u'CRSArrTime', u'UniqueCarrier',
       u'FlightNum', u'TailNum', u'ActualElapsedTime', u'CRSElapsedTime',
       u'AirTime', u'ArrDelay', u'DepDelay', u'Dest', u'Distance', u'TaxiIn',
       u'TaxiOut', u'Cancelled', u'CancellationCode', u'Diverted',
       u'CarrierDelay', u'WeatherDelay', u'NASDelay', u'SecurityDelay',
       u'LateAircraftDelay', u'lanes', u'origin_airport', u'dest_airport',
       u'lane_pairs', u'airport_y', u'city_y', u'dest_state', u'country_y',
       u'lat_y', u'long_y'],
      dtype='object')

In [9]:
# # first create a bar graph of the types of cancellations by type
# # the four categories are broken down as follows: A = carrier, B = weather, C = NAS, D = security
# c_code = {'CancellationCode':['A','B','C','D']}

# cancellations = flights['FlightNum'].groupby(flights['CancellationCode']).count()
# cancellations = pd.DataFrame(cancellations)
# cancellations.reset_index('CancellationCode', inplace = True)
# cancellations.rename(columns = {0:'cancellation_count'}, inplace = True)
# cancellations


In [10]:
# # calculate the average duration of each delay by type
# # first, carrier delays
# a_delay = flights['CarrierDelay'].mean()
# b_delay = flights['WeatherDelay'].mean()
# c_delay = flights['NASDelay'].mean()
# d_delay = flights['SecurityDelay'].mean()
# l_delay = flights['LateAircraftDelay'].mean()

# delay_dict = {'Carrier' : [a_delay], 
#               'Weather' : [b_delay], 
#               'NAS' : [c_delay], 
#               'Security' : [d_delay], 
#               'Late Plane' : [l_delay]
#              }

# delays = [a_delay, b_delay, c_delay, d_delay]
# delays = pd.DataFrame(delays)
# delays.rename(columns = {0:'avg_delay'}, inplace = True)
# delays['CancellationCode'] = ['A','B','C','D']
# # delays.set_index([['A','B','C','D']], inplace = True)
# # delays.reset_index()
# delays

In [11]:
# # now merge delay counts and average delay dataframes
# cancel_info = pd.merge(delays,cancellations, on = 'CancellationCode')
# cancel_info['category'] = ['Carrier','Weather','NAS','Security']
# cancel_info.set_index('CancellationCode', inplace = True)
# cancel_info.reset_index('CancellationCode', inplace = True)
# cancel_info.rename(columns = {'avg_delay':'avg_delay_min'}, inplace = True) # rename avg_delay to denote minutes
# cancel_info.head()

In [12]:
# # before graphing, set the category to the index for better xlabels
# cancel_info.set_index('category', inplace = True)
# cancel_info.cancellation_count.plot(kind = 'bar')
# plt.xlabel('Cancellation Code')
# plt.ylabel('Number of Flights')
# plt.title('Causes for Flight Cancellations in 2008')

In [13]:
# # since there is no record for cancellations due to late airplaces, we will have to measure the delays separately
# delays = pd.DataFrame.from_dict(delay_dict)
# delays = delays.T
# delays.rename(columns = {0:'avg_delay_min'}, inplace = True)
# delays.head()

In [14]:
# delays['avg_delay_min'].plot(kind = 'bar')
# plt.title('Causes for Flight Delays in 2008')
# plt.xlabel('Causes')
# plt.ylabel('Average Delay in Minutes')

In [15]:
# create a unique value (index-like) for the dataframe by resetting the index, then using that value as a key
# flights.reset_index(inplace = True)

In [16]:
# since counting the number of occurrences of each delay and keeping them in dataframes of the same length is a pain
# i'm going to do that another way: creating a column of 1s whenever there's an occurrence for each delay type

my_list = []

def delay_flag(series):
    for i in series:
        if math.isnan(i) == True or i == 0.0: 
        # use math module to determine if value is nan -> numpy doesn't necessarily work
        # use floats because the dtype is float64 (seen at the beginning)
            my_list.append(0) # return 0 if it's zero or nan
        else:
            my_list.append(1) # otherwise return 1 so we can count them later

In [23]:
# create flag variables for each delay type, then append them to the flights5 dataframe

my_list = []
delay_flag(flights5['CarrierDelay'])
flights5['f_carrier_delay'] = np.asarray(my_list)

my_list = []
delay_flag(flights5['WeatherDelay'])
flights5['f_weather_delay'] = np.asarray(my_list)

my_list = []
delay_flag(flights5['NASDelay'])
flights5['f_nas_delay'] = np.asarray(my_list)

my_list = []
delay_flag(flights5['SecurityDelay'])
flights5['f_sec_delay'] = np.asarray(my_list)

my_list = []
delay_flag(flights5['LateAircraftDelay'])
flights5['f_late_aircraft_delay'] = np.asarray(my_list)

In [24]:
flights5.columns

Index([u'Origin', u'airport_x', u'city_x', u'orig_state', u'country_x',
       u'lat_x', u'long_x', u'Year', u'Month', u'DayofMonth', u'DayOfWeek',
       u'DepTime', u'CRSDepTime', u'ArrTime', u'CRSArrTime', u'UniqueCarrier',
       u'FlightNum', u'TailNum', u'ActualElapsedTime', u'CRSElapsedTime',
       u'AirTime', u'ArrDelay', u'DepDelay', u'Dest', u'Distance', u'TaxiIn',
       u'TaxiOut', u'Cancelled', u'CancellationCode', u'Diverted',
       u'CarrierDelay', u'WeatherDelay', u'NASDelay', u'SecurityDelay',
       u'LateAircraftDelay', u'lanes', u'origin_airport', u'dest_airport',
       u'lane_pairs', u'airport_y', u'city_y', u'dest_state', u'country_y',
       u'lat_y', u'long_y', u'f_carrier_delay', u'f_weather_delay',
       u'f_nas_delay', u'f_sec_delay', u'f_late_aircraft_delay'],
      dtype='object')

In [25]:
# # change my_list to numpy array - TEST

# df = pd.DataFrame(np.random.random_integers(0,25,100))
# df.rename(columns = {0:'test'}, inplace = True)
# df['my_list'] = np.asarray(my_list)
# df.head()

In [26]:
# create lane pairs to determine highest traffic routes
# flights['lanes'] = flights['Origin'] + ' - ' + flights['Dest']

In [38]:
# group number of trips (using flight number) by lane pairs
lane_trips = flights5['FlightNum'].groupby(flights5['lane_pairs']).agg('count')
lane_trips = pd.DataFrame(lane_trips)
lane_trips.rename(columns = {0:'lane_trip_count'}, inplace = True)
lane_trips.reset_index(inplace = True)

print lane_trips.describe()
print ''
print lane_trips.head()

       lane_trip_count
count      5366.000000
mean       1306.322773
std        1593.373115
min           1.000000
25%         293.000000
50%         777.000000
75%        1718.500000
max       13788.000000

                                         lane_pairs  lane_trip_count
0  Abilene Regional-Dallas-Fort Worth International             2660
1          Adak-Ted Stevens Anchorage International              102
2         Adams -Baltimore-Washington International              366
3            Adams -Charlotte/Douglas International               39
4                             Adams -Chicago Midway              383


In [36]:
# create a stripped down of flights5 with desired columns to merge into lane_df_trips
flights6 = pd.DataFrame(flights5[['lane_pairs',
                     'origin_airport',
                     'orig_state',
                     'dest_airport',
                     'dest_state',]])

In [47]:
flights6.head()

,lane_pairs,origin_airport,orig_state,dest_airport,dest_state
0,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH
1,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH
2,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH
3,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH
4,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH


In [42]:
trips = flights5

In [44]:
# then group lane pairs by each type of delay
# i'm filtering the columns to exclude NaNs instead of values greater than 0 
# because 0 returns each series with a different length, making them difficult to merge

# first we'll do carrier delays
lane_df_carrier_delay = trips['f_carrier_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_carrier_delay = pd.DataFrame(lane_df_carrier_delay)
lane_df_carrier_delay.rename(columns = {0:'carrier_delays'}, inplace = True)
lane_df_carrier_delay.reset_index(inplace = True)
#print lane_df_carrier_delay.head()

# then we will do weather delays
lane_df_weather_delay = trips['f_weather_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_weather_delay = pd.DataFrame(lane_df_weather_delay)
lane_df_weather_delay.rename(columns = {0:'weather_delays'}, inplace = True)
lane_df_weather_delay.reset_index(inplace = True)
#lane_df_weather_delay.head()

# NAS delays
lane_df_nas_delay = trips['f_nas_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_nas_delay = pd.DataFrame(lane_df_nas_delay)
lane_df_nas_delay.rename(columns = {0:'nas_delays'}, inplace = True)
lane_df_nas_delay.reset_index(inplace = True)

# security delays
lane_df_sec_delay = trips['f_sec_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_sec_delay = pd.DataFrame(lane_df_sec_delay)
lane_df_sec_delay.rename(columns = {0:'security_delays'}, inplace = True)
lane_df_sec_delay.reset_index(inplace = True)

# late aircraft delays
lane_df_late_delay = trips['f_late_aircraft_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_late_delay = pd.DataFrame(lane_df_late_delay)
lane_df_late_delay.rename(columns = {0:'late_aircraft_delays'}, inplace = True)
lane_df_late_delay.reset_index(inplace = True)

print 'length of lane pairs:', len(lane_trips)
print 'length of carrier delays:', len(lane_df_carrier_delay)
print 'length of weather delays:', len(lane_df_weather_delay)
print 'length of NAS delays:', len(lane_df_nas_delay)
print 'length of late plane delays:', len(lane_df_late_delay)


length of lane pairs: 5366
length of carrier delays: 5366
length of weather delays: 5366
length of NAS delays: 5366
length of late plane delays: 5366


## Now that they're all the same length, we can combine them all with the lane pairs

In [73]:
# waterfall merge since pd.merge doesn't like more than merging two dataframes at once
df1 = pd.merge(lane_trips, lane_df_carrier_delay, on = 'lane_pairs')
df2 = pd.merge(df1, lane_df_weather_delay, on = 'lane_pairs')
df3 = pd.merge(df2, lane_df_nas_delay, on = 'lane_pairs')

# change names so they're easier to read
all_delays = pd.merge(df3, lane_df_late_delay, on = 'lane_pairs')
all_delays.rename(columns = {'lane_trip_count': 'trip_count',
                         'f_carrier_delay':'carrier',
                         'f_weather_delay':'weather',
                         'f_nas_delay':'NAS', 
                         'f_late_aircraft_delay':'late_aircraft'
                        }, inplace = True)
all_delays.head()


,lane_pairs,trip_count,carrier,weather,NAS,late_aircraft
0,Abilene Regional-Dallas-Fort Worth International,2660,145,31,161,107
1,Adak-Ted Stevens Anchorage International,102,5,0,7,20
2,Adams -Baltimore-Washington International,366,62,5,30,87
3,Adams -Charlotte/Douglas International,39,1,0,0,0
4,Adams -Chicago Midway,383,28,4,16,49


In [189]:
flights6.head()

,lane_pairs,origin_airport,orig_state,dest_airport,dest_state
0,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH
1,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH
2,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH
3,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH
4,Lehigh Valley International-Cleveland-Hopkins ...,Lehigh Valley International,PA,Cleveland-Hopkins Intl,OH


In [190]:
merged = pd.DataFrame(pd.merge(flights6, all_delays, on = 'lane_pairs', how = 'right'))

In [191]:
merged = merged.pivot_table(index=['lane_pairs','origin_airport','orig_state','dest_airport','dest_state'])
merged = merged.reset_index()
merged.ix[0:4]

,lane_pairs,origin_airport,orig_state,dest_airport,dest_state,NAS,carrier,late_aircraft,trip_count,weather
0,Abilene Regional-Dallas-Fort Worth International,Abilene Regional,TX,Dallas-Fort Worth International,TX,161,145,107,2660,31
1,Adak-Ted Stevens Anchorage International,Adak,AK,Ted Stevens Anchorage International,AK,7,5,20,102,0
2,Adams -Baltimore-Washington International,Adams,AR,Baltimore-Washington International,MD,30,62,87,366,5
3,Adams -Charlotte/Douglas International,Adams,AR,Charlotte/Douglas International,NC,0,1,0,39,0
4,Adams -Chicago Midway,Adams,AR,Chicago Midway,IL,16,28,49,383,4


## Now I have a dataframe that includes delay types by states that lets me analyze those delays at the state or airport level!

In [193]:
# i'll save this to a csv for further analysis - stay tuned!
merged.to_csv('datasets/flight_hist/cleaned_2008.csv')